<a href="https://colab.research.google.com/github/waveology/aire/blob/main/2_acceso_a_ficheros_de_datos_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# El acceso a datos almacenados en ficheros II

En este notebook vamos a empaquetar las instrucciones de preproceso de datos de fichero en una función para darle un uso más versátil.

###1. Definición de la función 
---

In [ ]:
def obtener_datos(anio=2023, magnitud=8, estacion=14, municipio=65) :

   import pandas as pd

   url = "https://raw.githubusercontent.com/waveology/aire/main/datos/comunidad"

   # Descarga los datos si no se han descargado previamente
   # ------------------------------------------------------------
   ![ ! -f {anio}.csv ] && wget {url}/{anio}.csv -O {anio}.csv

   # Lee el fichero de datos
   # -----------------------
   fichero = '%s.csv' % anio
   df = pd.read_csv(fichero,  
                 sep=';', 
                 decimal=',')
   # Filtramos por magnitud, estación y municipio
   # --------------------------------------------      
   df = df[ (df['magnitud']  ==  magnitud) 
       &    (df['estacion']  == estacion) 
       &    (df['municipio'] == municipio)]

   # Eliminamos columnas no necesarias
   # -----------------------------------
   df = df.drop(columns=['provincia','municipio','estacion','punto_muestreo','magnitud'])

   # Pasamos la hora que está en columnas a datos en filas
   # ---------------------------------------------------------
   df1 = df.melt(id_vars=['ano','mes','dia'],
                 value_vars = [ 'h%02d' % i for i in range(1,25)],
                 var_name='hora',
                 value_name='valor'
                 )

   # Convertimos la cadena de texto con el dato horario a valor numérico
   # --------------------------------------------------------------------
   df1['hora'] = df1['hora'].apply(lambda x : int(x[1:]))

   # Pasamos la validez que está en columnas a datos en filas
   # ---------------------------------------------------------
   df2 = df.melt(id_vars=['ano','mes','dia'],
                 value_vars = [ 'v%02d' % i for i in range(1,25)],
                 var_name='hora',
                 value_name='flag'
                 )

   # Convertimos la cadena de texto con la validez del dato a valor numérico
   # ------------------------------------------------------------------------   
   df2['hora'] = df2['hora'].apply(lambda x : int(x[1:]))

   # Fusionamos ambos dataframes
   # ----------------------------
   df = df1.merge(df2)

   # Eliminamos los datos no válidos y eliminamos la columna de validez
   # -------------------------------------------------------------------
   df = df[df['flag'] == 'V'].drop(columns='flag')

   # Creamos una columna para el tiempo
   # -----------------------------------
   df['fecha'] = pd.to_datetime({'year':df.ano,'month':df.mes,'day':df.dia,'hour':df.hora})

   # Elimninamos columnas no necesarias
   # -------------------------------------
   df = df.drop(columns=['ano','mes','dia','hora'])

   # Reordenamos las columnas (no es necesario)
   df = df[['fecha','valor']]

   # Establecemos el tiempo como índice
   # ------------------------------------
   df.set_index(['fecha'],inplace=True)

   # Ordenamos los datos por tiempo creciente
   df.sort_index(inplace=True)
   
   #Info
   print('Leídos %d datos' % len(df))
   
   return df

###2. Invocando la función
---

In [ ]:
anio = 2022
no2_getafe   = obtener_datos(anio=anio, magnitud=8, estacion=14, municipio=65)
no2_guadalix = obtener_datos(anio=anio, magnitud=8, estacion=1,  municipio=67)

In [ ]:
print(no2_getafe)
print(no2_guadalix)

Generamos graficos para ambos registros

In [ ]:
no2_getafe.plot(
        fontsize=12,
        figsize=(15,10),
        marker='o',
        ms=5,
        lw=1,
        grid=True,
        legend=False,
        title = 'Concentración de NO$_{2}$ ($\mu$g/m$^{3}$) medida en la estación de Getafe'
        )

no2_guadalix.plot(
        fontsize=12,
        figsize=(15,10),
        marker='o',
        ms=5,
        lw=1,
        grid=True,
        legend=False,
        title = 'Concentración de NO$_{2}$ ($\mu$g/m$^{3}$) medida en la estación de Guadalix de la Sierra'
        )


In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=(15,10))

ax.plot(no2_getafe.index, no2_getafe.valor, marker='o', ms=5, lw=1, color='blue',label='Getafe')
ax.plot(no2_guadalix.index, no2_guadalix.valor, marker='o', ms=5, lw=1, color='red',label='Guadalix')
ax.grid()
ax.legend()


In [ ]:
fig,ax = plt.subplots(2,1,figsize=(15,8))

ax[0].plot(no2_getafe.index, no2_getafe.valor, marker='o', ms=5, lw=1, color='blue')
ax[0].set_title('Getafe',size=18)
ax[1].plot(no2_guadalix.index, no2_guadalix.valor, marker='o', ms=5, lw=1, color='red')
ax[1].set_title('Guadalix',size=18)
plt.suptitle('Concentración de NO$_{2}$ ($\mu$g/m$^{3}$)  Año %s' % (anio),size=20)
for a in ax :
  a.grid(True)

In [ ]:
no2_getafe_2019   = obtener_datos(anio=2019, magnitud=8, estacion=14,  municipio=65)
no2_getafe_2020   = obtener_datos(anio=2020, magnitud=8, estacion=14,  municipio=65)



In [ ]:
fig,ax = plt.subplots(figsize=(15,10))

ax.plot(no2_getafe_2019.index, no2_getafe_2019.valor, marker='o', ms=5, lw=1, color='blue',label='2019')
ax.plot(no2_getafe_2020.index, no2_getafe_2020.valor, marker='o', ms=5, lw=1, color='red',label='2020')
ax.set_title('Concentración de NO$_{2}$ ($\mu$g/m$^{3}$) en Getafe')
ax.grid()
ax.legend()

In [ ]:
no2_mostoles_2019   = obtener_datos(anio=2019, magnitud=8, estacion=5,  municipio=92)
no2_mostoles_2020   = obtener_datos(anio=2020, magnitud=8, estacion=5,  municipio=92)

In [ ]:
fig,ax = plt.subplots(figsize=(15,10))

ax.plot(no2_mostoles_2019.index, no2_mostoles_2019.valor, marker='o', ms=5, lw=1, color='blue',label='2019')
ax.plot(no2_mostoles_2020.index, no2_mostoles_2020.valor, marker='o', ms=5, lw=1, color='red',label='2020')
ax.set_title('Concentración de NO$_{2}$ ($\mu$g/m$^{3}$) en Móstoles')
ax.grid()
ax.legend()